## Init

In [1]:
import torch
from torch.nn import functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
from tabulate import tabulate
from tqdm import tqdm, trange
from copy import deepcopy
import numpy as np
from collections import Counter

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

/home/konrad-kielczynski/miniconda3/envs/golem-ner/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Helper Functions

In [2]:
ALNUM_CHARSET = set('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
POLISH_CHARS = set('ąćęłńóśźżĄĆĘŁŃÓŚŹŻ')
ALNUM_CHARSET.update(POLISH_CHARS)

def convert_to_tokens(indices, tokenizer, extended=False, extra_values_pos=None, strip=True):
    if extended:
        res = [tokenizer.convert_ids_to_tokens([idx])[0] if idx < len(tokenizer) else 
               (f"[pos{idx-len(tokenizer)}]" if idx < extra_values_pos else f"[val{idx-extra_values_pos}]") 
               for idx in indices]
    else:
        res = tokenizer.convert_ids_to_tokens(indices)
    if strip:
        res = list(map(lambda x: x[1:] if x[0] == 'Ġ' else "#" + x, res))
    return res


def top_tokens(v, k=100, tokenizer=None, only_alnum=False, only_ascii=True, with_values=False, 
               exclude_brackets=False, extended=True, extra_values=None, only_from_list=None):
    if tokenizer is None:
        tokenizer = my_tokenizer
    v = deepcopy(v)
    ignored_indices = []
    if only_ascii:
        ignored_indices.extend([key for val, key in tokenizer.vocab.items() if not val.strip('Ġ▁').isascii()])
    if only_alnum: 
        ignored_indices.extend([key for val, key in tokenizer.vocab.items() if not (set(val.strip('Ġ▁[] ')) <= ALNUM_CHARSET)])
    if only_from_list:
        ignored_indices.extend([key for val, key in tokenizer.vocab.items() if val.strip('Ġ▁ ').lower() not in only_from_list])
    if exclude_brackets:
        ignored_indices = set(ignored_indices).intersection(
            {key for val, key in tokenizer.vocab.items() if not (val.isascii() and val.isalnum())})
        ignored_indices = list(ignored_indices)
        
    ignored_indices = list(set(ignored_indices))
    v[ignored_indices] = -np.inf
    extra_values_pos = len(v)
    if extra_values is not None:
        v = torch.cat([v, extra_values])
    values, indices = torch.topk(v, k=k)
    res = convert_to_tokens(indices, tokenizer, extended=extended, extra_values_pos=extra_values_pos)
    if with_values:
        res = list(zip(res, values.cpu().numpy()))
    return res

## Extract Weights

In [3]:
MODEL_ID = "sdadas/polish-gpt2-medium"

model = AutoModelForCausalLM.from_pretrained(MODEL_ID)
tokenizer = my_tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

In [4]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 1024)
    (wpe): Embedding(2048, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): FastGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=51200, bias=False)
)

In [12]:
model = AutoModelForCausalLM.from_pretrained(MODEL_ID)
tokenizer = my_tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
emb = model.get_output_embeddings().weight.data.T.detach()

num_layers = model.config.n_layer
num_heads = model.config.n_head
hidden_dim = model.config.n_embd
head_size = hidden_dim // num_heads

K = torch.cat([model.get_parameter(f"transformer.h.{j}.mlp.c_fc.weight").T
                           for j in range(num_layers)]).detach()
V = torch.cat([model.get_parameter(f"transformer.h.{j}.mlp.c_proj.weight")
                           for j in range(num_layers)]).detach()

W_Q, W_K, W_V = torch.cat([model.get_parameter(f"transformer.h.{j}.attn.c_attn.weight") 
                           for j in range(num_layers)]).detach().chunk(3, dim=-1)
W_O = torch.cat([model.get_parameter(f"transformer.h.{j}.attn.c_proj.weight") 
                           for j in range(num_layers)]).detach()


In [21]:
K_heads = K.reshape(num_layers, -1, hidden_dim)
V_heads = V.reshape(num_layers, -1, hidden_dim)
d_int = K_heads.shape[1]

W_Q_heads = W_Q.reshape(num_layers, hidden_dim, num_heads, head_size).permute(0, 2, 1, 3)
W_K_heads = W_K.reshape(num_layers, hidden_dim, num_heads, head_size).permute(0, 2, 1, 3)
W_V_heads = W_V.reshape(num_layers, hidden_dim, num_heads, head_size).permute(0, 2, 1, 3)
W_O_heads = W_O.reshape(num_layers, num_heads, head_size, hidden_dim)

In [22]:
W_Q_heads.shape

torch.Size([24, 16, 1024, 64])

In [23]:
emb_inv = emb.T
emb_inv.shape

torch.Size([51200, 1024])

## Interpretation

#### Alternative I: No Token List

In [24]:
tokens_list = set()

#### Alternative II: Can Load Token List from IMDB

In [25]:
from datasets import load_dataset

In [41]:
imdb = load_dataset('clarin-knext/wsd_polish_datasets')['train']['text']

In [42]:
max_tokens_num = None

In [59]:
if max_tokens_num is None:
    tokens_list = set()
    for txt in tqdm(imdb):
        tokens_list = tokens_list.union(set(tokenizer.tokenize(txt)))
else:
    tokens_list = Counter()
    for txt in tqdm(imdb):
        tokens_list.update(set(tokenizer.tokenize(txt)))
    tokens_list = map(lambda x: x[0], tokens_list.most_common(max_tokens_num))
    

100%|██████████| 7848/7848 [00:07<00:00, 1079.15it/s]


In [60]:
tokens_list = set([*map(lambda x: x.strip('Ġ▁').lower(), tokens_list)])

### FF Keys & Values

      Emb = (1024, 50257)

       (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          ...
       )

- projection of c_fc(key) -> c_key.T @ Emb
- projection of c_proj(value) -> c_value @ Emb


In [69]:
K_heads.shape

torch.Size([24, 4096, 1024])

In [70]:
K_heads[23, 907].shape

torch.Size([1024])

In [73]:
from tabulate import tabulate

def display_top_tokens(i1=23, i2=907):
    # i1, i2 = np.random.randint(num_layers), np.random.randint(d_int)

    print(i1, i2)
    print(tabulate([*zip(
        top_tokens((K_heads[i1, i2]) @ emb, k=30, only_from_list=tokens_list),
        top_tokens((V_heads[i1, i2]) @ emb, k=30, only_from_list=tokens_list),
        top_tokens((-K_heads[i1, i2]) @ emb, k=30, only_from_list=tokens_list),
        top_tokens((-V_heads[i1, i2]) @ emb, k=30, only_from_list=tokens_list),
    )], headers=['K', 'V', '-K', '-V']))

# Call the function
display_top_tokens()

23 907
K          V              -K          -V
---------  -------------  ----------  ---------
przycu     kody           dotychczas  #bot
zalog      #gory          rodzi       #lot
wylegi     #ei            do          #dzista
#walifik   #zmy           przebie     #remont
przesp     Apo            gatunku     #lee
pochowany  apokali        przez       #wan
#ppe       #128           #ja         #ette
wep        ludy           rodzin      #up
sfinans    #ords          zrazu       #bul
#iss       Cezary         lokalnie    #puszczam
#CS        archa          porywa      spu
#gny       przy           two         zamyka
#zwol      Homo           jeszcze     odstawi
#-).       litera         #jaw        #mont
lock       #pka           na          #beki
skonfisk   Benedykt       wymaga      #laks
#post      narodem        ty          tap
postoju    polityki       Drze        poby
erek       symbolu        pod         posto
#ionu      #lachet        Nie         #wana
#ppo       publicznego   

# służby porządkowe

In [74]:
display_top_tokens(12, 345)

12 345
K             V           -K      -V
------------  ----------  ------  ---------
ubraniu       kondu       #./     #berga
#czkom        #zyjny      #wiek   #CIA
przebra       ubrania     #An     #dge
ubiera        ubranie     #yn     #bul
#jazdy        munduru     #obie   #cznikiem
zakwater      mundury     #x      #dina
ubrania       spoczynku   #Bud    Steve
mundury       bieliz      ()      #CZY
ciuchy        #rowo       #Ener   #tp
wynajm        #cyjny      #wszy   #bergu
masek         elegancki   #ws     Chinami
skontak       #rysty      #zn     Lewis
ubranie       przepu      Jun     #cym
mieszkalnego  #lij        #Nu     #sonem
telef         nocleg      #stor   Jedwab
#ownikami     #anepi      Wen     #soci
#jazdem       schlu       #poty   #CI
pozory        fiska       #Gen    stad
spak          oznaczenia  #()     #ith
schroni       medyczna    #orga   Gary
zapak         biletu      #chia   Warren
#jazdu        mundur      #noza   #czo
ubraniach     aresztu     #owski  

# Instytucja, miasta, budulce

In [79]:
display_top_tokens(20, 4)

20 4


K            V           -K             -V
-----------  ----------  -------------  ----------
#Ubra        #Drugi      #ord           kamy
#Proto       #ros        #nem           kamieniu
musku        #nalny      #dal           marek
aminokwa     #owz        #siak          kamieniem
Sztuk        Poni        #co            kamieni
ubra         dziki       Rydzy          #latory
#Produkty    #ama        #ens           #kien
horm         Arnold      #ve            parasol
#mato        Boj         #anna          kamienic
#nospraw     Tob         Jawor          #kowca
wychowan     #nu         gminie         telewizo
sztan        #Sie        autostrady     walut
elasty       Wro         autostra       ce
#jet         #gust       Orange         #zione
Mecha        #ness       #dale          wideo
parami       #nad        tunelu         scen
genetycznie  #alizm      archidiecezji  #eu
#ship        #roe        parafii        #html
Mocy         duchem      dokument       kamieniami
dynamiki     

# imiona medycyna

In [81]:
i1, i2 = np.random.randint(num_layers), np.random.randint(d_int)

display_top_tokens(i1, i2)

10 3424
K               V           -K        -V
--------------  ----------  --------  ----------
#iry            #owaty      #kara     #loka
dech            #syw        #yta      pogro
#owz            #sista      poznamy   #lubie
#rk             #Wzrost     #zna      #ranki
#ott            #RL         #wizja    #ranka
#ard            #Kru        Kamila    #komo
parlamentarnej  #beta       #zera     pustymi
czynnej         #karzem     #dun      trium
#ine            #Lep        #nnik     #ryn
szklanych       #znaczne    #lada     #nity
mastur          #stem       #dem      #ion
Lake            #Inten      #chodem   #ionu
Fox             #Demon      #wid      #roczy
#ross           #start      #blin     #nium
bat             #Major      #styn     pia
palcami         #Mateusz    #lad      Ludowego
hazard          #Maksym     #cht      han
medytacji       #znania     #Mateusz  #ATO
baterii         #Bor        #gada     #obo
antykoncep      zbli        Bartosz   zamykam
#szczo          pie

### Attention Weights Interpretation

In [82]:
def approx_topk(mat, min_k=500, max_k=250_000, th0=10, max_iters=10, verbose=False):
    _get_actual_k = lambda th, th_max: torch.nonzero((mat > th) & (mat < th_max)).shape[0]
    th_max = np.inf
    left, right = 0, th0 
    while True:
        actual_k = _get_actual_k(right, th_max)
        if verbose:
            print(f"one more iteration. {actual_k}")
        if actual_k <= max_k:
            break
        left, right = right, right * 2
    if min_k <= actual_k <= max_k:
        th = right
    else:
        for _ in range(max_iters):
            mid = (left + right) / 2
            actual_k = _get_actual_k(mid, th_max)
            if verbose:
                print(f"one more iteration. {actual_k}")
            if min_k <= actual_k <= max_k:
                break
            if actual_k > max_k:
                left = mid
            else:
                right = mid
        th = mid
    return torch.nonzero((mat > th) & (mat < th_max)).tolist()

def get_top_entries(tmp, all_high_pos, only_ascii=False, only_alnum=False, exclude_same=False, exclude_fuzzy=False, tokens_list=None):
    remaining_pos = all_high_pos
    if only_ascii:
        remaining_pos = [*filter(
            lambda x: (tokenizer.decode(x[0]).strip('Ġ▁').isascii() and tokenizer.decode(x[1]).strip('Ġ▁').isascii()), 
            remaining_pos)]
    if only_alnum:
        remaining_pos = [*filter(
            lambda x: (tokenizer.decode(x[0]).strip('Ġ▁ ').isalnum() and tokenizer.decode(x[1]).strip('Ġ▁ ').isalnum()), 
            remaining_pos)]
    if exclude_same:
        remaining_pos = [*filter(
            lambda x: tokenizer.decode(x[0]).lower().strip() != tokenizer.decode(x[1]).lower().strip(), 
            remaining_pos)]
    if exclude_fuzzy:
        remaining_pos = [*filter(
            lambda x: not _fuzzy_eq(tokenizer.decode(x[0]).lower().strip(), tokenizer.decode(x[1]).lower().strip()), 
            remaining_pos)]
    if tokens_list:
        remaining_pos = [*filter(
            lambda x: ((tokenizer.decode(x[0]).strip('Ġ▁').lower().strip() in tokens_list) and 
                       (tokenizer.decode(x[1]).strip('Ġ▁').lower().strip() in tokens_list)), 
            remaining_pos)]

    pos_val = tmp[[*zip(*remaining_pos)]]
    good_cells = [*map(lambda x: (tokenizer.decode(x[0]), tokenizer.decode(x[1])), remaining_pos)]
    good_tokens = list(map(lambda x: Counter(x).most_common(), zip(*good_cells)))
    remaining_pos_best = np.array(remaining_pos)[torch.argsort(pos_val if reverse_list else -pos_val)[:50]]
    good_cells_best = [*map(lambda x: (tokenizer.decode(x[0]), tokenizer.decode(x[1])), remaining_pos_best)]
    # good_cells[:100]
    # list(zip(good_tokens[0], good_tokens[1]))
    return good_cells_best

#### $W_{VO}$ Interpretation

different heads capture different types of relations between pairs of vocabulary

Choose **layer** and **head** here:

In [101]:
i1, i2 = np.random.randint(num_layers), np.random.randint(num_heads)
i1, i2 = 23, 9
i1, i2

(23, 9)

In [102]:
W_V_heads.shape, W_O_heads.shape

(torch.Size([24, 16, 1024, 64]), torch.Size([24, 16, 64, 1024]))

In [103]:
W_V_tmp, W_O_tmp = W_V_heads[i1, i2, :], W_O_heads[i1, i2]
tmp = (emb_inv @ (W_V_tmp @ W_O_tmp) @ emb)

In [104]:
all_high_pos = approx_topk(tmp, th0=1, verbose=True) # torch.nonzero((tmp > th) & (tmp < th_max)).tolist()

one more iteration. 0
one more iteration. 0
one more iteration. 87
one more iteration. 16925


In [105]:
exclude_same = False
reverse_list = False
only_ascii = True
only_alnum = False

kodeks przypis [, ]

In [106]:
get_top_entries(tmp, all_high_pos, only_ascii=only_ascii, only_alnum=only_alnum, 
                exclude_same=exclude_same, tokens_list=None)

[('].', '['),
 ('],', '['),
 (' ).', ' ('),
 (']', '['),
 ('...),', ' ('),
 ('przyp', ' ('),
 (' ),', ' ('),
 ('-).', ' ('),
 ('ZOBACZ', ' ['),
 ('zob', ' ('),
 ('przyp', ' ['),
 ('.]', '['),
 ('%)', ' ('),
 ('!),', ' ('),
 ('Ash', ' ('),
 ('].', ' ['),
 ('.].', '['),
 ('],', ' ['),
 (' rezygnacja', ' ('),
 ('!).', ' ('),
 ('prem', ' ('),
 ('\x15', ' ('),
 ('...),', ' ("'),
 ('orom', ' ('),
 ('%),', ' ('),
 ('ecie', ' ['),
 ('tul', '['),
 ('.].', ' ['),
 ('CZYTAJ', ' ['),
 ('etu', ' ['),
 ('-)', ' ('),
 ('przyp', ' ("'),
 ('%).', ' ('),
 ('?),', ' ('),
 ('sic', ' ['),
 ('http', ' ('),
 (' ]', ' ['),
 ('Dzwonek', ' ('),
 (' ).', ' ("'),
 ('!)', ' ('),
 ('demon', ' ('),
 (' realizacja', ' ('),
 ('dlaczego', ' ('),
 ('.]', ' ['),
 ('Dzwonek', ' ("'),
 ('Kodeks', ' ['),
 ('Bir', ' ['),
 ('Ash', ' ("'),
 ('eu', ' ('),
 ('przyp', ' (+')]

In [107]:
def w_vo(i1, i2, only_ascii=True, only_alnum=False, exclude_same=False, exclude_fuzzy=False, tokens_list=None):
    W_V_tmp, W_O_tmp = W_V_heads[i1, i2, :], W_O_heads[i1, i2]
    tmp = (emb_inv @ (W_V_tmp @ W_O_tmp) @ emb)
    all_high_pos = approx_topk(tmp, th0=1, verbose=True) # torch.nonzero((tmp > th) & (tmp < th_max)).tolist()
    return get_top_entries(tmp, all_high_pos, only_ascii=only_ascii, only_alnum=only_alnum, 
                           exclude_same=exclude_same, exclude_fuzzy=exclude_fuzzy, tokens_list=tokens_list)

chcecie wasze nasze opisujące osby

In [158]:
i1, i2 = 20, 4

w_vo(i1, i2, only_ascii=True, only_alnum=False, exclude_same=False, exclude_fuzzy=False, tokens_list=None)

one more iteration. 0
one more iteration. 1
one more iteration. 15804


[(' chcecie', ' waszych'),
 (' chcecie', ' waszym'),
 (' musicie', ' waszych'),
 (' chcecie', ' wam'),
 (' waszych', ' waszych'),
 (' chcecie', ' chcecie'),
 (' musicie', ' waszym'),
 ('ujecie', ' waszych'),
 (' chcecie', ' waszego'),
 (' musicie', ' musicie'),
 (' musicie', ' wam'),
 (' chcecie', ' waszej'),
 (' chcecie', ' was'),
 (' chcecie', ' wasze'),
 (' chcecie', ' wami'),
 (' znacie', ' waszych'),
 ('ujecie', ' waszym'),
 (' musicie', ' waszego'),
 (' macie', ' waszych'),
 (' musicie', ' waszej'),
 (' musicie', ' wami'),
 (' wasze', ' waszych'),
 (' chcecie', 'ujecie'),
 (' musicie', ' was'),
 (' chcecie', 'czycie'),
 (' znacie', ' waszym'),
 ('ujecie', ' wam'),
 (' waszych', ' waszym'),
 (' musicie', ' wasze'),
 (' widzicie', ' waszych'),
 (' chcecie', 'dzicie'),
 (' waszych', ' wam'),
 (' chcecie', ' musicie'),
 ('ujecie', ' was'),
 (' chcecie', ' zobaczycie'),
 ('ujecie', ' waszego'),
 (' chcecie', ' wasz'),
 (' robicie', ' waszych'),
 (' waszym', ' waszych'),
 (' rozumiecie

prezydent, lewica, rakiety

In [160]:
i1, i2 = 10, 15

w_vo(i1, i2, only_ascii=True, only_alnum=False, exclude_same=False, exclude_fuzzy=False, tokens_list=None)

one more iteration. 0
one more iteration. 0
one more iteration. 0
one more iteration. 0
one more iteration. 75015


[(' jach', 'nomor'),
 (' samochodowy', ' kierowcy'),
 ('kowiak', ' wojewoda'),
 ('owiak', ' wojewoda'),
 ('owiak', ' wojewody'),
 (' Prezydent', 'ckiewicz'),
 ('Prezydent', 'ckiewicz'),
 (' samochodowy', ' pojazdu'),
 ('mady', 'hady'),
 (' jacht', 'nomor'),
 ('Trump', ' konstytucyjne'),
 (' lotnicze', ' piloci'),
 (' morska', 'hire'),
 (' chleb', ' zmywar'),
 (' rolne', ' gospodarstwach'),
 (' kontrah', ' wyceny'),
 ('loe', 'drow'),
 (' jach', ' Navy'),
 (' przeciwpancer', 'pancer'),
 ('stadt', ' Poniat'),
 ('Porucznik', ' Poniat'),
 (' auta', ' kierowcy'),
 (' rowerowe', ' kierowcy'),
 ('dun', 'hire'),
 ('woje', 'hady'),
 (' rakiet', 'pancer'),
 (' helikopter', ' lotnictwa'),
 (' rolne', ' gospodarstwa'),
 ('szenko', ' powiat'),
 (' helikopter', ' samolocie'),
 (' Renault', ' wagony'),
 (' rolne', ' Gospodarstwa'),
 (' rolnego', ' Gospodarstwa'),
 ('sfak', ' NFZ'),
 (' samochodowych', ' kierowcy'),
 (' floty', ' Navy'),
 (' bmw', ' kierowcy'),
 ('Tadeusz', ' Poniat'),
 ('rek', ' stype

okresla polozenie

In [161]:
i1, i2 = 21, 7

w_vo(i1, i2, only_ascii=True, only_alnum=False, exclude_same=False, exclude_fuzzy=False, tokens_list=None)

one more iteration. 0
one more iteration. 0
one more iteration. 10
one more iteration. 5182


[(' ranem', ' nad'),
 ('ornie', ' przez'),
 ('datek', ' nad'),
 ('arl', ' przed'),
 (' pewno', ' na'),
 ('spodziewanie', ' nad'),
 (' razu', ' od'),
 ('miernie', ' nad'),
 (' okazji', ' przy'),
 (' wsk', ' na'),
 (' czele', ' na'),
 ('orne', ' przez'),
 ('godziny', ' nad'),
 ('sione', ' przed'),
 (' ranem', 'nad'),
 ('natural', ' nad'),
 ('przewo', ' nad'),
 (' Duna', ' nad'),
 ('ktory', ' do'),
 ('miar', ' nad'),
 (' dobra', ' dla'),
 (' Jezi', ' nad'),
 ('spodzie', ' nad'),
 (' niedawna', ' do'),
 ('pisie', ' pod'),
 (' wygody', ' dla'),
 ('arcie', ' przed'),
 (' sumie', ' w'),
 ('miernie', 'nad'),
 ('tek', ' pod'),
 ('wcze', ' przed'),
 ('mier', ' nad'),
 ('hala', ' pod'),
 ('ornie', ' przeze'),
 (' uboczu', ' na'),
 (' podstawie', ' na'),
 (' ranem', 'Nad'),
 ('czesne', ' do'),
 ('granicznych', ' nad'),
 ('wiska', ' przez'),
 (' barkach', ' na'),
 ('ornie', 'przez'),
 (' odmiany', ' dla'),
 ('niego', ' przed'),
 (' plecami', ' za'),
 (' dobi', ' na'),
 ('przewodni', ' nad'),
 (' ko

In [185]:
i1, i2 = 22, 9

w_vo(i1, i2, only_ascii=True, only_alnum=True, exclude_same=False, exclude_fuzzy=False, tokens_list=None)

one more iteration. 0
one more iteration. 0
one more iteration. 173
one more iteration. 34601


[(' trucizny', ' truci'),
 (' nowotwor', ' onko'),
 (' trucizny', ' trucizny'),
 (' AIDS', ' AIDS'),
 (' astro', ' astro'),
 (' immuno', ' immuno'),
 (' trup', ' trup'),
 (' HIV', ' AIDS'),
 (' gazy', ' gazy'),
 (' radioaktyw', ' radioaktyw'),
 (' grypy', ' szczepion'),
 (' przeszcze', ' przeszcze'),
 (' wirusem', ' wirus'),
 (' wirusem', ' wirusa'),
 (' epidem', ' epidem'),
 (' truci', ' truci'),
 (' peni', ' peni'),
 (' trup', ' trupa'),
 (' Wetery', ' Wetery'),
 (' HIV', ' HIV'),
 (' szczepionki', ' szczepionki'),
 (' ofierze', ' ofierze'),
 (' prochu', ' prochu'),
 (' grypy', ' grypy'),
 (' czaszki', ' czaszki'),
 ('zdrowi', 'zdrowi'),
 (' szczepionki', ' szczepion'),
 (' uzdrowi', ' uzdrowi'),
 (' grypy', ' szczepionki'),
 (' grypy', ' wirus'),
 (' geode', ' geode'),
 (' nowotwor', ' nowotw'),
 (' ofierze', ' ofiara'),
 (' raka', ' onko'),
 (' onko', ' onko'),
 (' strzyka', ' strzyka'),
 (' trupa', ' trup'),
 (' wetery', ' wetery'),
 (' epidem', ' epidemio'),
 (' wirusem', ' wirus

imiona

In [189]:
i1, i2 = 20, 7

w_vo(i1, i2, only_ascii=True, only_alnum=True, exclude_same=False, exclude_fuzzy=False, tokens_list=None)

one more iteration. 0
one more iteration. 5
one more iteration. 68762


[('gla', ' Doug'),
 ('sarzu', ' Jones'),
 ('gla', ' Dou'),
 ('losci', ' Danny'),
 ('shire', ' Tommy'),
 ('anom', ' Fabi'),
 ('tetu', ' Violet'),
 ('ach', ' Josh'),
 ('aryjnych', ' Romu'),
 ('dzu', ' Amanda'),
 ('orki', ' Davis'),
 ('sarzu', ' Jone'),
 ('dalem', ' Ryszar'),
 ('lip', ' Phil'),
 ('ranki', ' Piot'),
 ('nictw', ' Molly'),
 ('lotte', ' Charlotte'),
 ('oty', ' Cole'),
 ('nold', ' Rey'),
 ('fina', ' Leo'),
 ('osci', ' Mitch'),
 ('ic', ' Artem'),
 ('gla', ' Douglas'),
 ('ftu', ' Ellie'),
 ('ach', ' Gam'),
 ('derlan', ' Molly'),
 ('del', ' Bogdan'),
 ('nim', ' Anto'),
 ('alie', ' Nathan'),
 ('tyfika', ' Violet'),
 ('st', ' Cami'),
 ('rence', ' Violet'),
 ('rzu', ' Waw'),
 ('prawy', ' Szymon'),
 ('nowskim', ' Marty'),
 ('tyny', ' Alber'),
 ('osc', ' Mitch'),
 ('ngu', ' Lyn'),
 ('toli', ' Feliks'),
 ('stia', ' Seba'),
 ('aryjnych', ' Catherine'),
 ('lett', ' Scar'),
 ('isa', ' Lou'),
 ('tto', ' Lance'),
 ('tanu', ' Ryan'),
 ('lomet', ' Agata'),
 ('ria', ' Jeremy'),
 ('fus', ' Vale

#### $W_{QK}$ Interpretation

Q - Query, K - Key co na co odpowiada

Choose **layer** and **head** here:

In [116]:
# i1, i2 = np.random.randint(num_layers), np.random.randint(num_heads)
i1, i2 = 20, 13
i1, i2

(20, 13)

In [117]:
W_Q_tmp, W_K_tmp = W_Q_heads[i1, i2, :], W_K_heads[i1, i2, :]
tmp2 = (emb_inv @ (W_Q_tmp @ W_K_tmp.T) @ emb_inv.T)

In [118]:
all_high_pos = approx_topk(tmp2, th0=1, verbose=True) # torch.nonzero((tmp2 > th2) & (tmp2 < th_max2)).tolist()

one more iteration. 0
one more iteration. 0
one more iteration. 755


In [119]:
exclude_same = False
reverse_list = False
only_ascii = True
only_alnum = True

In [152]:
get_top_entries(tmp2, all_high_pos, only_ascii=only_ascii, only_alnum=only_alnum, exclude_same=exclude_same, 
                tokens_list=None)

[(' waszych', ' wasze'),
 (' waszych', ' twoje'),
 (' waszych', ' twoi'),
 (' waszych', ' twoimi'),
 (' jakbym', 'mam'),
 (' waszym', ' wasze'),
 (' waszych', ' waszych'),
 (' waszych', ' twoich'),
 (' naszych', ' twoje'),
 (' waszego', ' wasze'),
 (' waszych', ' wasza'),
 (' waszych', ' twoim'),
 (' naszych', ' wasze'),
 (' Waszej', ' twoje'),
 (' waszych', ' wasz'),
 (' waszym', ' twoje'),
 (' wasze', ' wasze'),
 (' waszej', ' wasze'),
 (' waszych', ' twojej'),
 (' wasza', ' wasze'),
 (' waszego', ' twoje'),
 (' Waszej', ' wasze'),
 (' waszego', ' twoi'),
 (' naszymi', ' wasze'),
 (' wasz', ' wasze'),
 (' wasz', ' waszych'),
 (' naszych', ' twoich'),
 (' naszych', ' twoimi'),
 (' waszych', ' waszym'),
 (' waszym', ' twoi'),
 (' waszego', ' wasza'),
 (' waszej', ' twoje'),
 (' naszych', ' Twoje'),
 (' waszych', ' twoja'),
 (' wasza', ' twoje'),
 (' waszym', ' waszych'),
 (' naszymi', ' twoje'),
 (' wasze', ' twoje'),
 (' waszych', ' Twoje'),
 (' Waszej', ' twojej'),
 (' wasz', ' twoi'

In [153]:
def w_qk(i1, i2, only_ascii=True, only_alnum=False, exclude_same=False, exclude_fuzzy=False, tokens_list=None):
    W_Q_tmp, W_K_tmp = W_Q_heads[i1, i2, :], W_K_heads[i1, i2, :]
    tmp2 = (emb_inv @ (W_Q_tmp @ W_K_tmp.T) @ emb_inv.T)
    all_high_pos = approx_topk(tmp2, th0=1, verbose=True) # torch.nonzero((tmp2 > th2) & (tmp2 < th_max2)).tolist()
    return get_top_entries(tmp2, all_high_pos, only_ascii=only_ascii, only_alnum=only_alnum, 
                           exclude_same=exclude_same, exclude_fuzzy=exclude_fuzzy, tokens_list=None)

In [155]:
i1, i2 = 9, 5

w_qk(i1, i2, only_ascii=True, only_alnum=False, exclude_same=False, exclude_fuzzy=False, tokens_list=tokens_list)

one more iteration. 0
one more iteration. 11
one more iteration. 399
one more iteration. 43253


[('").', '").'),
 ('").', '"),'),
 ('").', '")'),
 ('"),', '").'),
 ('")', '").'),
 ('")', '")'),
 ('"),', '")'),
 ('")', '"),'),
 ('"),', '"),'),
 ('").', ' ).'),
 ('").', '.).'),
 ('").', ');'),
 (')?', '")'),
 (')?', '").'),
 (')?', '"),'),
 ('").', ').'),
 ('").', ')?'),
 (');', '").'),
 ('").', '.)'),
 ('").', '%).'),
 ('").', '?).'),
 ('")', ');'),
 ('"),', ')?'),
 ('")', ')?'),
 ('").', ' ),'),
 ('"),', ');'),
 ('?).', '").'),
 (');', '"),'),
 ('").', '!).'),
 ('!).', '").'),
 ('").', '.),'),
 ('?)', '")'),
 ('):', '")'),
 ('")', '.)'),
 ('").', '...).'),
 ('?).', '"),'),
 ('").', '%)'),
 ('):', '").'),
 ('?)', '"),'),
 ('):', '"),'),
 ('?).', '")'),
 ('"),', ' ).'),
 ('").', '?)'),
 ('")', '?)'),
 ('").', '?),'),
 ('").', '),'),
 ('").', '):'),
 ('"),', '.)'),
 ('?)', '").'),
 ('"),', ' ),')]

organizacje

In [157]:
i1, i2 = 0, 7

w_qk(i1, i2, only_ascii=True, only_alnum=False, exclude_same=False, exclude_fuzzy=False, tokens_list=None)

one more iteration. 0
one more iteration. 0
one more iteration. 13437


[(' funduszu', ' Schengen'),
 (' fundusz', ' UEFA'),
 (' klucza', 'Samolot'),
 (' klucza', ' lotniczej'),
 (' fundusz', ' Schengen'),
 (' jednostki', ' Stoczni'),
 (' klucza', ' lotnicze'),
 (' funduszu', ' EURO'),
 (' Sojuszu', ' NATO'),
 (' fundusz', ' KRUS'),
 (' jednostki', ' stoczni'),
 (' nos', ' lotniczy'),
 (' fundusz', ' EURO'),
 (' sieci', ' ryb'),
 (' fundusz', ' OFE'),
 (' wody', ' podwodnych'),
 (' klucza', ' lotniczy'),
 (' sieci', ' ryba'),
 (' Sojusz', ' NATO'),
 (' funduszu', ' UEFA'),
 (' DJ', ' dywizji'),
 (' klucza', ' lotniczych'),
 (' klucza', ' Lotnictwa'),
 (' klucza', ' lotniczego'),
 (' wody', ' ryba'),
 (' Luf', ' 1943'),
 (' konty', ' NATO'),
 (' fundusz', ' UE'),
 (' organizacji', ' Schengen'),
 (' wody', ' zatoki'),
 (' Fron', ' ZSRR'),
 (' MW', ' Marynar'),
 (' nos', ' lotniczego'),
 (' jednostek', ' Stoczni'),
 (' wody', ' ryby'),
 (' klucza', ' samolot'),
 (' mostek', ' kaju'),
 (' funduszu', ' UE'),
 (' zrzu', ' 1944'),
 (' nos', ' samolocie'),
 (' klu

Pis, sejm

In [167]:
i1, i2 = 7, 9

w_qk(i1, i2, only_ascii=True, only_alnum=True, exclude_same=False, exclude_fuzzy=False, tokens_list=None)

one more iteration. 0
one more iteration. 0
one more iteration. 149
one more iteration. 162255


[('leen', 'isen'),
 ('gebjorg', 'owiska'),
 ('encji', 'zitel'),
 ('mera', 'ssen'),
 (' MINIST', ' MINIST'),
 ('gis', 'mowski'),
 ('gebjorg', 'isen'),
 ('anna', 'jewskiego'),
 ('encji', 'leran'),
 (' PIS', 'ktywi'),
 (' Departam', ' MINIST'),
 (' terapeu', 'kompen'),
 ('uszko', 'szowi'),
 ('encji', 'ktywi'),
 ('encji', 'bin'),
 ('lotu', 'szewskiego'),
 (' polityczny', ' MINIST'),
 (' PiSu', 'ktywi'),
 (' Koali', 'niesiono'),
 ('encji', 'nia'),
 (' rozowepaski', 'czniku'),
 (' akademi', ' szczy'),
 (' Narodowa', ' MINIST'),
 ('COP', 'niesiono'),
 ('gebjorg', 'randa'),
 (' rozowepaski', 'lomet'),
 ('encji', 'oll'),
 (' KRRiT', 'ktywi'),
 ('lecz', 'dar'),
 ('lette', 'ssen'),
 ('gebjorg', 'jew'),
 ('encji', 'pnie'),
 ('lette', 'szewski'),
 ('encji', 'gini'),
 ('gebjorg', 'nowu'),
 ('anna', 'szewskiego'),
 ('lamin', 'ssen'),
 (' lgbt', 'rzmi'),
 ('lette', 'rance'),
 ('gy', 'niesione'),
 ('lecz', 'obra'),
 ('ggy', 'dnicki'),
 ('gebjorg', 'niew'),
 (' rozowepaski', 'dne'),
 ('jeb', 'dar'),
 ('

podroze

In [169]:
i1, i2 = 22, 5

w_qk(i1, i2, only_ascii=True, only_alnum=True, exclude_same=False, exclude_fuzzy=False, tokens_list=None)

one more iteration. 0
one more iteration. 1
one more iteration. 2021


[(' pieszo', ' pieszo'),
 (' jechali', ' pieszo'),
 (' dojazd', ' pieszo'),
 (' spaceru', ' pieszo'),
 (' jech', ' pieszo'),
 (' wsiada', ' pieszo'),
 (' przemieszczania', ' pieszo'),
 (' space', ' pieszo'),
 (' maszer', ' pieszo'),
 (' pieszych', ' pieszo'),
 (' przemieszcza', ' pieszo'),
 (' jedzie', ' pieszo'),
 (' przyjechali', ' pieszo'),
 (' spacer', ' pieszo'),
 (' trasy', ' pieszo'),
 (' spacery', ' pieszo'),
 (' pieszego', ' pieszo'),
 (' rozpak', ' rozpak'),
 (' trasa', ' pieszo'),
 (' przecha', ' pieszo'),
 (' stacjon', ' stacjon'),
 (' przyjedzie', ' pieszo'),
 (' pieszy', ' pieszo'),
 (' trasie', ' pieszo'),
 (' wysiada', ' pieszo'),
 (' korytarz', ' schodami'),
 (' przewie', ' pieszo'),
 (' pieszo', ' rowerem'),
 (' wyprawa', ' pieszo'),
 (' drzwi', ' drzwi'),
 (' przetransport', ' pieszo'),
 ('jechali', ' pieszo'),
 (' rowerem', ' pieszo'),
 (' autokar', ' pieszo'),
 (' konno', ' pieszo'),
 (' wioz', ' pieszo'),
 ('jazdu', ' pieszo'),
 (' pulpi', ' pulpi'),
 ('kilomet', 

przeklenstwa

In [171]:
i1, i2 = 23, 9

w_qk(i1, i2, only_ascii=True, only_alnum=True, exclude_same=False, exclude_fuzzy=False, tokens_list=None)

one more iteration. 0
one more iteration. 179
one more iteration. 4992


[(' T', 'day'),
 (' N', 'jscy'),
 (' chuj', ' chuj'),
 (' etc', ' etc'),
 (' itp', ' etc'),
 (' H', 'alla'),
 ('pierdo', ' jeb'),
 (' H', 'HA'),
 (' R', 'rene'),
 (' M', 'taria'),
 (' R', 'sung'),
 (' N', 'iers'),
 (' itp', ' itp'),
 (' N', 'Obiekt'),
 (' M', 'wychw'),
 (' N', ' ktorzy'),
 (' p', 'jpg'),
 (' chuj', ' jeb'),
 (' P', 'znesu'),
 (' etc', ' itp'),
 (' M', 'sztof'),
 (' L', ' Gospodarczej'),
 (' F', 'Pilot'),
 (' N', ' przyklad'),
 (' G', 'Gri'),
 (' chuja', ' chuj'),
 (' M', 'stora'),
 (' kurwa', ' jeb'),
 (' itd', ' etc'),
 (' T', 'ciarze'),
 (' J', ' OPowi'),
 (' l', 'Dzwonek'),
 (' M', 'chium'),
 (' P', 'omosci'),
 (' kurwa', ' pierdol'),
 (' R', ' Szkolnictwa'),
 (' R', 'VID'),
 ('pierdol', ' jeb'),
 (' chuja', ' jeb'),
 ('pierdala', ' chuj'),
 (' J', 'zawodni'),
 (' M', 'cjusza'),
 (' kurwa', ' chuj'),
 (' jeb', ' jeb'),
 (' R', 'stur'),
 ('jeb', ' chuj'),
 (' D', 'Kal'),
 (' F', 'hem'),
 (' N', 'cznosci'),
 (' J', 'ROW')]

ilosci

In [172]:
i1, i2 = 19, 1

w_qk(i1, i2, only_ascii=True, only_alnum=True, exclude_same=False, exclude_fuzzy=False, tokens_list=None)

one more iteration. 0
one more iteration. 0
one more iteration. 334
one more iteration. 353457
one more iteration. 3623


[(' ani', ' ani'),
 (' ani', ' zadnej'),
 (' ani', ' zadnego'),
 (' zadnego', ' ani'),
 (' komukolwiek', ' komukolwiek'),
 (' zadnej', ' ani'),
 (' dowolny', ' dowolnie'),
 (' dowolnie', ' dowolnie'),
 (' ani', ' zadnych'),
 (' dowolne', ' dowolnie'),
 (' dowolnej', ' dowolnie'),
 (' komukolwiek', ' kogokolwiek'),
 (' dowolnym', ' dowolnie'),
 (' zadnych', ' ani'),
 (' zadnego', ' zadnego'),
 (' kogokolwiek', ' kogokolwiek'),
 (' kogokolwiek', ' komukolwiek'),
 (' jakiejkolwiek', ' komukolwiek'),
 (' jakimkolwiek', ' komukolwiek'),
 (' zadnego', ' zadnej'),
 (' dowolnego', ' dowolnie'),
 (' dowolnej', ' dowolnej'),
 (' jakiegokolwiek', ' komukolwiek'),
 (' ani', ' nigdzie'),
 (' drugiemu', ' drugiemu'),
 (' kiedykolwiek', ' kiedykolwiek'),
 (' czegokolwiek', ' czegokolwiek'),
 (' jakimkolwiek', ' kogokolwiek'),
 (' czegokolwiek', ' komukolwiek'),
 (' drugie', ' druga'),
 (' drugiemu', ' obu'),
 (' czegokolwiek', ' kogokolwiek'),
 (' jakichkolwiek', ' ani'),
 (' dowolne', ' dowolne'),
 

zwroty osobowe

In [174]:
i1, i2 = 18, 5

w_qk(i1, i2, only_ascii=True, only_alnum=True, exclude_same=False, exclude_fuzzy=False, tokens_list=None)

one more iteration. 0
one more iteration. 0
one more iteration. 4606


[(' Waszej', ' pan'),
 (' Twoim', ' Pana'),
 (' twoim', ' pana'),
 (' twoim', ' Pana'),
 (' twoim', ' Panu'),
 (' twoim', ' pan'),
 (' Twoim', ' pana'),
 (' twoim', ' panie'),
 (' Twoim', ' pan'),
 (' Twoim', ' panie'),
 (' Waszej', ' Wasza'),
 (' Twoim', ' Pan'),
 (' Waszej', ' Pan'),
 (' waszym', ' pan'),
 (' Waszej', ' szanow'),
 (' tobie', ' pan'),
 (' Twoim', ' szanow'),
 (' Twoim', ' wasz'),
 (' Waszej', ' pana'),
 (' waszym', ' pana'),
 (' ciebie', ' Panu'),
 (' wam', ' pan'),
 (' Ciebie', ' Pana'),
 (' Waszej', ' Pana'),
 (' tobie', ' Panu'),
 (' Twoim', ' Panu'),
 (' tobie', ' pana'),
 (' Waszej', ' pani'),
 (' waszym', ' Pana'),
 (' twoim', ' panienka'),
 (' ciebie', ' pan'),
 (' Ciebie', ' Pan'),
 (' Wam', ' pan'),
 (' Twojej', ' Pana'),
 (' wasz', ' pan'),
 (' Twoim', ' Wasza'),
 (' twoimi', ' pana'),
 (' twoimi', ' pan'),
 (' twoim', ' panu'),
 (' twoim', ' Pan'),
 (' Wam', ' Pan'),
 (' waszym', ' wasz'),
 (' ciebie', ' Pana'),
 (' tobie', ' Pana'),
 (' waszym', ' Panu'),


sporty

In [175]:
i1, i2 = 21, 12

w_qk(i1, i2, only_ascii=True, only_alnum=True, exclude_same=False, exclude_fuzzy=False, tokens_list=None)

one more iteration. 0
one more iteration. 0
one more iteration. 1063


[(' wnim', ' oczym'),
 (' wtym', ' oczym'),
 (' turniej', ' turnieju'),
 (' turnieju', ' turnieju'),
 (' futbol', ' futbol'),
 (' znim', ' oczym'),
 (' tenis', ' futbol'),
 (' o', 'zwykopem'),
 (' film', ' filmowej'),
 (' ztego', ' oczym'),
 (' kolar', ' futbol'),
 (' lekkoatle', ' zawodnicy'),
 (' wnim', ' apotem'),
 (' wtym', ' Jego'),
 (' wdomu', ' ale'),
 (' kolar', ' zawodnicy'),
 (' filmowej', ' filmowej'),
 (' aktor', ' aktor'),
 (' turniej', ' turniej'),
 (' sport', ' zawodnicy'),
 (' film', ' filmowe'),
 (' wmo', ' oczym'),
 (' wtym', ' apotem'),
 (' zawodni', ' zawodnicy'),
 (' kibi', ' zawodnicy'),
 (' o', 'noscia'),
 (' wtej', ' oczym'),
 (' znim', ' apotem'),
 (' wna', ' oczym'),
 (' PZPN', ' turnieju'),
 (' wjego', ' Jego'),
 (' wdomu', ' oczym'),
 (' tenis', ' mecz'),
 (' tenis', ' zawodnicy'),
 (' kolar', ' sprin'),
 (' wnim', ' czego'),
 (' film', ' serialu'),
 (' muzycznym', ' muzycznej'),
 (' kolar', ' Tour'),
 (' film', ' filmowy'),
 (' muzyki', ' muzycznej'),
 (' s